In [3]:
import wiggum as wg
import pandas as pd
import numpy as np
import timeit
import seaborn as sns
import warnings
from itertools import chain
warnings.filterwarnings('ignore')

In [4]:
final=pd.DataFrame()

def test_scalability(labeled_df):

    times=[]
    for command in commands:
        for pearson_obj in objs:

            statement=command.format(pearson_obj)
            print(statement)
            t=timeit.repeat(statement, prereqs, repeat=10, number=100)
            times.append(t)
            print(t)
    timed = [ele for lis in times for ele in lis]
    return timed

def create_df(N, number_cluster, num_splitby, labeled_df, num_dep=None, num_indep=None, num_dep_indep=None):
    timings=test_scalability(labeled_df)
    function_names=['cct', 'get_subgroup_trends', 'get_trends (agg)', 'get_trends (sub)']
    result_df=pd.DataFrame(columns=['times', 'data', 'function'])
    # setting function column
    function=[]
    for ele in function_names:
        function.extend([ele]*20)
    # setting version column
    version=[['v1']*10, ['v2']*10]*len(function_names)
    version=list(chain.from_iterable(version))
    # setting trial column
    trial=list(range(10))*(len(function_names)*2)

    data_name='synthetic {}'.format(N)
    
    result_df['times']=timings
    result_df['function']=function
    result_df['data']=str(data_name)
    result_df['trial']=trial
    result_df['version']=version
    result_df['number_cluster']=number_cluster
    result_df['num_splitby']=num_splitby
    if num_dep_indep==None:
        result_df['independent']=num_indep
        result_df['dependent']=num_dep
    else:
        result_df['dep_indep_vars']=num_dep_indep
        
    return result_df


In [5]:
import mlsim
from mlsim import sp_plot

In [6]:
np.random.seed(20210627)


r_clusters =     [-.8, .5, .4, .7, -.6, .5, .4,-.9]  # magnitude correlation coefficient of clusters
cluster_spread = [ .3,-.2, .1,-.4, .2, -.1, .2,.4] # pearson correlation of means
p_sp_clusters =  [  1,  0,  0,  0,  1,   0,  0,  1] # portion of clusters with SP  #1 if r neg 0 if rpos
cluster_size =   [2,  3]#
domain_range = [0, 20, 0, 20] # of all data

In [5]:
n_list = [1000, 5000, 10000]
number_cluster_list=[2, 4, 8, 16, 32]
num_dep_indep_list = [4, 8, 16]
num_splitby_list = [4, 8, 16]

In [12]:
# generate data
for N in n_list:
    for number_cluster in number_cluster_list:
        for num_dep_indep in num_dep_indep_list:
            for num_splitby in num_splitby_list:
                k = [number_cluster]*40
                n_view = len(k)
                p_clusters = [[1/k_i]*k_i for k_i in k]
                many_sp_df = mlsim.geometric_indep_views_gmm_sp(n_view,r_clusters,cluster_size,cluster_spread,p_sp_clusters,
                            domain_range,k,N,p_clusters)
                # print(many_sp_df.shape)
                labeled_df = wg.LabeledDataFrame(many_sp_df)
                
                n_view = int(len(labeled_df.df.columns) / 3)

                # set dependent and independent for some xi, ignore for the rest
                dep_indep_list = np.random.choice(n_view*2, num_dep_indep)
                #print(dep_indep_list)
                roles = {'x'+str(i+1):['ignore'] if i not in dep_indep_list else ['independent','dependent'] 
                            for i in range(n_view*2)}

                # vars without 'x' in them are splitbys
                splitby_var_list = [cn for cn in labeled_df.df.columns if not('x' in cn)]
                # set splitby for some variable, ignore for the rest
                splitby_list = np.random.choice(splitby_var_list, num_splitby)
                roles.update( {c:['splitby'] if c in splitby_list else ['ignore'] for c in splitby_var_list})    

                count_list = []

                var_types = {'x'+str(i+1):'continuous' for i in range(n_view*2)}
                var_types.update( {c:'categorical' for c in splitby_var_list})
                weighting = {}

                labeled_df.set_counts(count_list)
                labeled_df.set_roles(roles)
                labeled_df.set_var_types(var_types)
                labeled_df.meta_df
                
                print('N:', N)
                print('Number of clusters:', number_cluster)
                print('Number of dep indep vars:', num_dep_indep)
                print('Number of sliptby vars:', num_splitby)
                
                result_df=create_df(N, num_dep_indep, number_cluster, num_splitby)
                final=pd.concat([final, result_df])
                del labeled_df



N: 5000
Number of clusters: 2
Number of dep indep vars: 4
Number of sliptby vars: 4
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.3607352000000219, 0.320321300000046, 0.310594100000003, 0.3078242000000273, 0.3189363000000185, 0.3074754000000439, 0.31456920000005084, 0.3150163999999904, 0.3869733000000224, 0.36477530000001934]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.31916729999989, 0.35681529999999384, 0.40308570000001964, 0.39563889999999446, 0.40087449999998626, 0.3793122999999241, 0.3843717000000879, 0.37638590000005934, 0.38574849999997696, 0.4095012999999881]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.629111499999908, 1.8885291999999936, 1.8500917999999729, 1.565436200000022, 1.4103406000001542, 2.18812099999991, 1.3865743000001203, 1.6321749000001091, 1.5353920000000016, 1.3943429000000833]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.460756100000026, 1.4177142999999433, 1.40242760000001, 1.4098344999999881, 1.43477589999

[0.8349505999999565, 0.8117045999999846, 0.8070617000000766, 0.8133920999998736, 0.820162400000072, 0.8252335999998195, 1.3504794000000402, 1.4047821999999996, 1.002007299999832, 0.9495448000000124]
N: 5000
Number of clusters: 2
Number of dep indep vars: 8
Number of sliptby vars: 8
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.5575630999999248, 0.5093664000000899, 0.5152537999999822, 0.5192026999998234, 0.5044585000000552, 0.5117448999999397, 0.5105356999999913, 0.5146466999999575, 0.5029078999998546, 0.5035864999999831]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.26799239999991187, 0.25189530000011473, 0.2667010999998638, 0.26071460000002844, 0.2605260999998791, 0.2531149999999798, 0.26286019999997734, 0.2624759999998787, 0.2611939999999322, 0.25603650000016387]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.832949699999972, 1.4357465999999022, 1.443173999999999, 1.4694848999999977, 1.5371420999999827, 1.4329979999999978, 1.4811495999999806

[0.8341711999999006, 0.8181111000001238, 0.8233700000000681, 0.8250027999999929, 0.8129272000001038, 0.816960999999992, 0.8334683000000496, 0.8225607999997919, 0.8062989999998535, 0.8260271999999986]
N: 5000
Number of clusters: 2
Number of dep indep vars: 16
Number of sliptby vars: 16
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[1.130695199999991, 1.2358120999999755, 1.111195700000053, 1.1262132000001657, 1.1299432999999226, 1.1093439999999646, 1.1719874999998865, 1.1027662000001328, 1.1072219000000132, 1.097107699999924]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.27174489999993057, 0.2522459000001618, 0.2517737999999099, 0.2508241000000453, 0.2663121000000501, 0.2591701000001194, 0.25663250000002336, 0.24911629999996876, 0.26243270000009034, 0.2525225000001683]
labeled_df.get_subgroup_trends_1lev([pearson])
[3.1014213999999356, 1.4618739999998525, 1.4880298999999013, 1.4695550999999796, 1.4764001999999437, 1.4633627000000615, 1.460883199999898

[0.8200056000000586, 0.8264137000001028, 0.8035175999998501, 0.7970804999999928, 0.980796900000314, 0.838679499999671, 0.8231065999998464, 0.810002600000189, 0.8100847000000613, 0.8110664999999244]
N: 5000
Number of clusters: 4
Number of dep indep vars: 8
Number of sliptby vars: 4
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.3781493000001319, 0.330943699999807, 0.24954489999981888, 0.21338539999987916, 0.23285759999998845, 0.21418050000011135, 0.21131070000001273, 0.2093030999999428, 0.21311430000014298, 0.22865920000003825]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.27518849999978556, 0.252038900000116, 0.2469756000000416, 0.26223049999998693, 0.26148830000011003, 0.25398939999968206, 0.24911030000021128, 0.2667514999998275, 0.2784183999997367, 0.3209170999998605]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.913803800000096, 1.3867794000002505, 1.3916581000003134, 1.3822105999997802, 1.4597054999999273, 1.3818077999999332, 1.38800050000

[0.8428475999999137, 0.813033699999778, 0.8396667000001798, 0.810958699999901, 0.8061122999997679, 0.8041828000000351, 0.8173965999999382, 0.8114906999999221, 0.8065486000000419, 0.7966208999996525]
N: 5000
Number of clusters: 4
Number of dep indep vars: 16
Number of sliptby vars: 8
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[1.257984000000306, 1.2506846000001133, 1.3971198999997796, 1.5044284999999036, 1.5114785999999185, 1.5189331000001403, 1.4634954000002836, 1.4747710999999981, 1.426517899999908, 1.244142600000032]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.26197959999990417, 0.24959230000013122, 0.2787220000000161, 0.3955063000003065, 0.893493000000035, 0.25035479999996824, 0.25428580000016154, 0.2569058000003679, 0.258691799999724, 0.2501121999998759]
labeled_df.get_subgroup_trends_1lev([pearson])
[3.2560692000001836, 1.4866164000000026, 1.4958473000001504, 1.4827519999998913, 1.4578118999997969, 1.4953513999998904, 1.474395799999911, 1.

...

[0.8299850000003062, 0.8175989999999729, 0.8223280000001978, 0.7985727999998744, 0.8208998000000065, 0.8490882000000965, 0.8388866999998754, 0.8109008000001268, 0.8072204000000056, 0.9900594000000638]
N: 5000
Number of clusters: 16
Number of dep indep vars: 4
Number of sliptby vars: 8
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.27549520000002303, 0.2567834000001312, 0.2616491000003407, 0.2738453000001755, 0.26291700000001583, 0.33375200000000405, 0.3806744999997136, 0.27095120000012685, 0.26395830000001297, 0.2568185000000085]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.2738273999998455, 0.26364990000001853, 0.25748079999993934, 0.25501959999974133, 0.2716700999999375, 0.25833920000013677, 0.2480291000001671, 0.2507838999999876, 0.2576038000001972, 0.25577430000021195]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.0340989999999692, 1.4123509000000922, 1.4305282000000261, 1.4412343999997574, 1.4180876000000353, 1.4423658000000614, 1.405266

[0.8652262999994491, 0.8177494999999908, 0.8273000999997748, 0.8298248999999487, 0.8283562999995411, 0.8377485000000888, 0.8205385000001115, 0.8411501000000499, 0.8360941000000821, 0.8266878000003999]
N: 5000
Number of clusters: 16
Number of dep indep vars: 8
Number of sliptby vars: 16
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.8480878000000303, 1.0301546999999118, 1.0237092999996094, 1.112188299999616, 1.114511400000083, 0.8703042000006462, 0.9406103000001167, 0.9615524999999252, 0.5647877000001245, 0.5243650000002162]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.27091199999995297, 0.27148849999957747, 0.2699520999995002, 0.25526799999988725, 0.31728190000012546, 0.33691910000015923, 0.34946560000025784, 0.31036690000019007, 0.2548663999996279, 0.36661180000010063]
labeled_df.get_subgroup_trends_1lev([pearson])
[5.527088499999991, 1.5709233999996286, 1.5814614000000802, 1.5603167999997822, 1.566986700000598, 1.564447900000232, 1.572305300000

[0.8622074000004432, 0.8345836999997118, 0.8271455000003698, 0.8285334999991392, 0.8249795000001541, 0.8131290999999692, 0.8262650000006033, 0.7955646000000343, 0.820992799999658, 0.8283463999996457]
N: 5000
Number of clusters: 32
Number of dep indep vars: 4
Number of sliptby vars: 4
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[1.398452599999473, 1.4107033999998748, 1.3994570000004387, 1.508784800000285, 1.5089097000000038, 1.5137186000001748, 1.4304212999995798, 1.390903200000139, 1.384689299999991, 1.3951922999995077]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.26204890000008163, 0.2541090999993685, 0.24717940000027738, 0.277358400000594, 0.32273780000014085, 0.2727414000000863, 0.27922990000024583, 0.2632347999997364, 0.257999600000403, 0.2559776999996757]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.064364399999249, 1.443255500000305, 1.4180919999998878, 1.4567164000000048, 1.4114564999999857, 1.4238844000001336, 1.4500538000002052, 1.4

[0.8489650999999867, 0.8271750000003522, 0.8213581999998496, 0.8376483000001826, 0.8192072000001644, 0.81181849999939, 0.8317784000000756, 0.8130037000000812, 0.8238332999999329, 0.846007200000713]
N: 5000
Number of clusters: 32
Number of dep indep vars: 8
Number of sliptby vars: 8
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.5055829000002632, 0.5025366999998369, 0.4998153000005914, 0.5082535999999891, 0.5078432000000248, 0.49464050000005955, 0.4934693999994124, 0.5021667999999408, 0.5044903999996677, 0.4983476999996128]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.2650635999998485, 0.24958650000007765, 0.26083440000002156, 0.26146889999927225, 0.25980600000002596, 0.25074839999979304, 0.2544578000006368, 0.26482840000062424, 0.2579571999995096, 0.2555191000001287]
labeled_df.get_subgroup_trends_1lev([pearson])
[5.419325899999421, 1.5416946999994252, 1.5492270999993707, 1.5734928000001673, 1.545038199999908, 1.5964953000002424, 1.88517120000051

[0.8676515000006475, 0.8310920000003534, 0.8426505999996152, 0.8339295000005222, 0.8143966999996337, 0.8538820000003398, 0.8807627000005596, 0.8305116000001362, 0.82417710000027, 0.8306006000002526]
N: 5000
Number of clusters: 32
Number of dep indep vars: 16
Number of sliptby vars: 16
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[1.1492864000001646, 1.2598110999997516, 1.1851441999997405, 1.1231336999999257, 1.097325599999749, 1.1091678000002503, 1.1041100000002189, 1.1020846000001256, 1.1167273999999452, 1.0994791000002806]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.2610068999993018, 0.24732550000044284, 0.26709480000045005, 0.2558835999998337, 0.2511606000007305, 0.2484279999998762, 0.2634507999991911, 0.26085909999983414, 0.3160684000004039, 0.28274080000028334]
labeled_df.get_subgroup_trends_1lev([pearson])
[25.149144799999704, 2.724837199999456, 2.550408199999765, 2.9943702000000485, 2.2386397000000215, 2.1957727999997587, 2.217329699999936

In [147]:
final.to_csv('data/final_data')

In [22]:
df_1000=data[data['data']=='synthetic 1000']
df_10000=data[data['data']=='synthetic 10000']

In [24]:
final_result=pd.concat([df_1000, final, df_10000])

In [26]:
final_result.to_csv('data/final_data')

In [6]:
N = 1000
number_cluster = 2
num_dep_indep = 4
num_splitby = 4

In [7]:
def generate_data(N, number_cluster, num_splitby, num_dep=None, num_indep=None, num_dep_indep=None):   
    k = [number_cluster]*40
    n_view = len(k)
    p_clusters = [[1/k_i]*k_i for k_i in k]
    many_sp_df = mlsim.geometric_indep_views_gmm_sp(n_view,r_clusters,cluster_size,cluster_spread,p_sp_clusters,
                domain_range,k,N,p_clusters)
    # print(many_sp_df.shape)
    labeled_df = wg.LabeledDataFrame(many_sp_df)

    n_view = int(len(labeled_df.df.columns) / 3)

    roles={'x1':['ignore']}
    if num_dep_indep==None:
        i=0
        while i<(n_view*2):
            if i<indep:
                roles.update({'x'+str(i+1):['independent']})
            elif i>indep and i<(indep+dep):
                roles.update({'x'+str(i+1):['dependent']})
            else:
                roles.update({'x'+str(i+1):['ignore']})
            i+=1
    else:
        # set dependent and independent for some xi, ignore for the rest
        dep_indep_list = np.random.choice(n_view*2, num_dep_indep)
        #print(dep_indep_list)
        roles = {'x'+str(i+1):['ignore'] if i not in dep_indep_list else ['independent','dependent'] 
                    for i in range(n_view*2)}
    # vars without 'x' in them are splitbys
    splitby_var_list = [cn for cn in labeled_df.df.columns if not('x' in cn)]
    # set splitby for some variable, ignore for the rest
    splitby_list = np.random.choice(splitby_var_list, num_splitby)
    roles.update( {c:['splitby'] if c in splitby_list else ['ignore'] for c in splitby_var_list})    

    count_list = []

    var_types = {'x'+str(i+1):'continuous' for i in range(n_view*2)}
    var_types.update( {c:'categorical' for c in splitby_var_list})
    weighting = {}

    labeled_df.set_counts(count_list)
    labeled_df.set_roles(roles)
    labeled_df.set_var_types(var_types)
    labeled_df.meta_df
    
    return labeled_df

In [19]:
result=pd.DataFrame()
indep=2
objs=['pearson', 'pearson_2']
prereqs='from __main__ import labeled_df, pearson, pearson_2, objs, commands'
commands=["{}.compute_correlation_table(labeled_df.df, 'agg_trend')",
          'labeled_df.get_subgroup_trends_1lev([{}])',
         "{}.get_trends(labeled_df.df, 'agg_trend')",
         "{}.get_trends(labeled_df.df, 'sub_trend')"]

while indep<=20:
    dep=2
    while dep<=20: 
        labeled_df=generate_data(N, number_cluster, dep, indep, num_splitby)
        pearson = wg.All_Pearson()
        pearson.get_trend_vars(labeled_df)

        pearson_2 = wg.trends.All_Pearson_V2()
        pearson_2.get_trend_vars(labeled_df)
        
        result_df=create_df(N, number_cluster, num_splitby, labeled_df, num_dep=dep, num_indep=indep, num_dep_indep=None)
        result=pd.concat([result, result_df])
        print('independents: ', indep)
        print('dependents: ', dep)
        dep+=2
    indep+=2

pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.1903464999923017, 0.17124239998520352, 0.1770550000073854, 0.17393250000895932, 0.16557270000339486, 0.16598619997967035, 0.18475129999569617, 0.15933330002008006, 0.16768620000220835, 0.1814652000030037]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.15457939999760129, 0.18042119999881834, 0.16428479997557588, 0.18616740001016296, 0.16981890000170097, 0.24099089999799617, 0.17118209999171086, 0.18085750000318512, 0.20166069999686442, 0.18227020002086647]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.9566683000011835, 1.7267058000143152, 1.8958043000020552, 1.591501299990341, 2.0300108999945223, 2.1253652999876067, 2.2092952000093646, 2.023269799974514, 1.7459649000084028, 2.0483091000060085]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.785276500013424, 1.8939330999855883, 1.9478392000019085, 2.104343000013614, 2.0006895999831613, 1.9928174000233412, 2.072403999976814, 1.94869240000844, 1.88

pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.28364050001255237, 0.25809280000976287, 0.27342129999306053, 0.275423499988392, 0.2622360999812372, 0.2681284999998752, 0.25906419998500496, 0.2531147999980021, 0.2704915000067558, 0.2744028000161052]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.3206715999986045, 0.37147310000727884, 0.2685790999967139, 0.3582773000234738, 0.32027759999618866, 0.2996976000140421, 0.2924526999995578, 0.2956871000060346, 0.29629699999350123, 0.298264700017171]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.8662794000119902, 1.7858343000116292, 1.64692090000608, 1.6816471000202, 1.7832839000038803, 1.8185368000122253, 1.8835811999742873, 1.9146577000210527, 2.027598900021985, 1.6498102999757975]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.8122670000011567, 1.543785099987872, 1.6202508000133093, 1.6518737000005785, 1.591445300000487, 1.8196819000004325, 1.6779264000069816, 1.6538842000009026, 1.707482800004072

[0.5010887000244111, 0.4738229000067804, 0.49411780000082217, 0.4600026000116486, 0.4777730999921914, 0.4706976999877952, 0.44177570001920685, 0.507385200005956, 0.5626446999958716, 0.5930127000028733]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.638104600016959, 0.44049120001727715, 0.5274185000162106, 0.408978899999056, 0.44820029998663813, 0.44776069998624735, 0.46954759999061935, 0.4464480000024196, 0.4704030999855604, 0.4241213999921456]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.0482053000014275, 1.8631754000089131, 2.0580750000081025, 1.9031220999895595, 2.0893781000049785, 2.4857707000046503, 2.071859699994093, 1.8057504000025801, 1.994090199994389, 2.1511270999908447]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.9631515000073705, 1.779023299983237, 2.0900547000055667, 1.8298727000073995, 1.9678983000048902, 1.7312141000002157, 1.6584180000063498, 1.7397928000136744, 2.017848099989351, 1.8317261000047438]
pearson.get_trends(labeled_df.df, 'ag

[0.3050848999992013, 0.24426730000413954, 0.24852459999965504, 0.23432479999610223, 0.233094399998663, 0.24257109998143278, 0.27032179999514483, 0.23227089998545125, 0.3290450999920722, 0.25992509999196045]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.27320870000403374, 0.2342773000127636, 0.25440940001863055, 0.23975730000529438, 0.23420889998669736, 0.2341415999981109, 0.23438390000956133, 0.2568186999997124, 0.24226100000669248, 0.23221330001251772]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.8300291000050493, 1.589856900012819, 1.5120180000085384, 1.6063856999971904, 1.6290120000194293, 1.6319031000020914, 1.6805122999940068, 1.6438518000068143, 1.5253397000196856, 1.6304462999978568]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.915073800017126, 1.5698989999946207, 1.5846578999771737, 1.7753365999960806, 1.7644109999819193, 1.8650207000027876, 1.8361302000121213, 1.7899502999789547, 1.6357245999970473, 1.6169776000024285]
pearson.get_trends(labele

[0.46322790000704117, 0.42258670000592247, 0.40077169999131, 0.39062610000837594, 0.3966813999868464, 0.40031850000377744, 0.40072659999714233, 0.4419136999931652, 0.3935159999819007, 0.45441269999719225]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.4016018999973312, 0.3958312000031583, 0.4717250000103377, 0.4164736999955494, 0.42625829999451526, 0.3981705000041984, 0.4227135000110138, 0.4379194999928586, 0.3953620999818668, 0.4209998999722302]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.9725428999809083, 1.6027584999974351, 1.5379880999971647, 1.5953264000127092, 1.5423387000046205, 1.6282000000064727, 1.6372676999890245, 1.737786599987885, 1.5001477999903727, 1.5457328999764286]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.5855609999853186, 1.6628106999851298, 1.5958736999891698, 1.5726965000212658, 1.7281604000017978, 1.6711778999888338, 1.6296829000057187, 1.57919230000698, 1.5880576000199653, 1.5868346999923233]
pearson.get_trends(labeled_df.df, 

...

[0.7748477999994066, 0.773650599992834, 0.771000400010962, 0.766322500014212, 0.761344900005497, 0.7568342000013217, 0.7867825999855995, 0.7540084999927785, 0.7792361999745481, 0.7617793999961577]
pearson_2.get_trends(labeled_df.df, 'sub_trend')
[1.99325480000698, 1.9835480000183452, 1.978070100012701, 1.9716743000026327, 2.00406129998737, 2.0009617999894544, 1.9984605000063311, 2.0489369000133593, 1.9617840999853797, 1.9692216999828815]
independents:  18
dependents:  4
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.6429652999795508, 0.6434195000038017, 0.6489201000076719, 0.6398480000207201, 0.6256404999876395, 0.6371826999820769, 0.6840785999957006, 0.635118500009412, 0.6297934000031091, 0.6375073000090197]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.6406675000034738, 0.641736999998102, 0.6279212000081316, 0.6334905000112485, 0.6252700999903027, 0.6155318999954034, 0.6417889000149444, 0.6280982999887783, 0.6374209999921732, 0.6434020000160672]

[6.589191100007156, 6.570344899984775, 6.547788099996978, 6.565859800000908, 6.538571700017201, 6.540544400020735, 6.531250800006092, 7.351761500001885, 7.544148799992399, 6.903547099995194]
independents:  18
dependents:  12
pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[1.148202099982882, 1.1405079000105616, 1.1423194999806583, 1.1531322000082582, 1.1841943000035826, 1.1652269999904092, 1.1596965999924578, 1.2278065999853425, 1.144972099980805, 1.1395861000055447]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[1.144932199997129, 1.1534866000001784, 1.1315239999967162, 1.1277758000069298, 1.1594316000118852, 1.1480895999993663, 1.1651693000167143, 1.154833900014637, 1.14293830000679, 1.1443192999868188]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.4419934999896213, 1.484703800000716, 1.4765826000075322, 1.5034403000026941, 1.4707342000037897, 1.5158457000216004, 1.5390420999901835, 1.5025355000107083, 1.462317700003041, 1.4747340999892913]
labeled

pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.45047739997971803, 0.44624379999004304, 0.4412939999892842, 0.43457030001445673, 0.44133019997389056, 0.43555699998978525, 0.44153119999100454, 0.4312323999765795, 0.4299759999848902, 0.4443846000067424]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.4573333000007551, 0.4483284000016283, 0.43947869999101385, 0.44728339998982847, 0.4386975000088569, 0.43177640001522377, 0.43890619999729097, 0.44040510000195354, 0.4566493000020273, 0.4401176000246778]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.6171718999976292, 1.4293979999783915, 1.440797899995232, 1.4920739000081085, 1.6970143000071403, 1.7578680999868084, 1.7422173999948427, 1.7839718999748584, 1.800548399973195, 1.5214415000227746]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.4872961000073701, 1.4622154000098817, 1.4594407000113279, 1.4561016999941785, 1.4598716000036802, 1.4498818000138272, 1.4534426000027452, 1.4589340999955311, 1.433

[1.0370731999864802, 0.9815470000030473, 0.9774494000012055, 0.9933758999977726, 0.9894948000146542, 1.0037411999946926, 0.971834999974817, 0.9800156999845058, 0.9806372000020929, 0.9669121000042651]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.9887456000142265, 0.9860389000095893, 0.976075899990974, 0.9689906999992672, 0.9809104000160005, 0.9874873000080697, 0.9830035000049975, 0.9671181999729015, 0.9720807000121567, 0.9770401999994647]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.067789900000207, 1.4929336000059266, 1.5009465999901295, 1.4668764999951236, 1.4944291999854613, 1.5064818999962881, 1.500026199995773, 1.4789102999784518, 1.490354399982607, 1.5049576999736018]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.5110777999798302, 1.5079265999956988, 1.4845838000182994, 1.4944221999903675, 1.469587200001115, 1.4881783000018913, 1.4893915000138804, 1.475070099986624, 1.515957299998263, 1.461075900006108]
pearson.get_trends(labeled_df.df, 'agg_trend'

[1.5639389999851119, 1.553383299993584, 1.54827440000372, 1.5357059999951161, 1.5444413000077475, 1.5440006999997422, 1.6206634999834932, 1.5546156000054907, 1.5936318000021856, 1.5746705000055954]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.433099499990931, 1.4726093000208493, 1.4834107999922708, 1.4658612000057474, 1.4707347999792546, 1.4707179999968503, 1.4742529999930412, 1.5085432000050787, 1.9047657999908552, 1.9668155999970622]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.8252140000113286, 1.4828306999988854, 1.4882211999793071, 1.4802775999996811, 1.485403700004099, 1.7797505999915302, 2.0461737999867182, 1.9392028999864124, 1.9930145000107586, 1.942070799996145]
pearson.get_trends(labeled_df.df, 'agg_trend')
[2.016347800003132, 1.9115313999936916, 1.910532100009732, 1.893287600018084, 1.8826188999810256, 1.91125489998376, 1.9234227999986615, 1.8921397999802139, 1.8974236999929417, 1.8767211000085808]
pearson_2.get_trends(labeled_df.df, 'agg_trend')
[11.00381840000

In [31]:
indep=2
dep=2
labeled_df=generate_data(N, number_cluster, dep, indep, num_splitby)
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
x1,float64,continuous,NaN,False,NaN
x2,float64,continuous,[independent],False,NaN
x3,float64,continuous,[independent],False,NaN
x4,float64,continuous,[dependent],False,NaN
x5,float64,continuous,[dependent],False,NaN
...,...,...,...,...,...
JJ,object,categorical,[ignore],False,NaN
KK,object,categorical,[ignore],False,NaN
LL,object,categorical,[ignore],False,NaN


In [20]:
result

,times,data,function,trial,version,independent,dependent
0,0.190346,synthetic 1000,cct,0,v1,2,2
1,0.171242,synthetic 1000,cct,1,v1,2,2
2,0.177055,synthetic 1000,cct,2,v1,2,2
3,0.173933,synthetic 1000,cct,3,v1,2,2
4,0.165573,synthetic 1000,cct,4,v1,2,2
...,...,...,...,...,...,...,...
75,12.362285,synthetic 1000,get_trends (sub),5,v2,20,20
76,13.186259,synthetic 1000,get_trends (sub),6,v2,20,20
77,13.927814,synthetic 1000,get_trends (sub),7,v2,20,20
78,12.258961,synthetic 1000,get_trends (sub),8,v2,20,20


In [21]:
result.to_csv('data/vars_data')

In [41]:
del labeled_df

In [17]:
n_list = [1000, 5000]
number_cluster_list=[2, 4, 8, 16, 32]
num_indep_list = [5, 10, 15, 20, 25]
num_dep_list = [5, 10, 15, 20, 25]
num_splitby_list = [4, 8, 16]

In [8]:
n_list = [1000]
number_cluster_list=[8, 16, 32]
num_indep_list = [5, 10, 15, 20, 25]
num_dep_list = [5, 10, 15, 20, 25]
num_splitby_list = [4, 8, 16]

In [18]:
result=pd.DataFrame()
indep=5
objs=['pearson', 'pearson_2']
prereqs='from __main__ import labeled_df, pearson, pearson_2, objs, commands'
commands=["{}.compute_correlation_table(labeled_df.df, 'agg_trend')",
          'labeled_df.get_subgroup_trends_1lev([{}])',
         "{}.get_trends(labeled_df.df, 'agg_trend')",
         "{}.get_trends(labeled_df.df, 'sub_trend')"]

for N in n_list:
    for number_cluster in number_cluster_list:
        for dep in num_dep_list:
            for indep in num_indep_list:
                for num_splitby in num_splitby_list: 
                    labeled_df=generate_data(N, number_cluster, dep, indep, num_splitby)
                    pearson = wg.All_Pearson()
                    pearson.get_trend_vars(labeled_df)

                    pearson_2 = wg.trends.All_Pearson_V2()
                    pearson_2.get_trend_vars(labeled_df)

                    result_df=create_df(N, number_cluster, num_splitby, labeled_df, num_dep=dep, num_indep=indep, num_dep_indep=None)
                    result=pd.concat([result, result_df])
                    print('independents: ', indep)
                    print('dependents: ', dep)


pearson.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.38229759997921064, 0.24927659999229945, 0.24833210001816042, 0.24755390000063926, 0.25723309998284094, 0.24729129998013377, 0.259121499984758, 0.31670549997943453, 0.3111260000150651, 0.24965220000012778]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.25996609998401254, 0.24918099999194965, 0.2677160000021104, 0.27184670002316125, 0.25448509998386726, 0.24019069998757914, 0.293495999998413, 0.2602861000050325, 0.26911600001039915, 0.2456751000136137]
labeled_df.get_subgroup_trends_1lev([pearson])
[2.104596599994693, 1.6586008000012953, 1.7119342000223696, 2.0656592000159435, 1.5676284999935888, 1.4817018999892753, 1.486874700000044, 1.5215687000018079, 1.9196506000007503, 1.5964002000109758]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.6334555999783333, 1.882971599989105, 1.872485300002154, 1.7420882000005804, 1.8547028999892063, 1.5315433999930974, 1.683408100012457, 1.5547185000032187, 1.52697

[0.36848400000599213, 0.361188700015191, 0.36015900000347756, 0.35225050000008196, 0.3529452999937348, 0.37148939998587593, 0.35208609999972396, 0.3499249000160489, 0.36431820000871085, 0.3800352000107523]
pearson_2.compute_correlation_table(labeled_df.df, 'agg_trend')
[0.36921229999279603, 0.37818470000638627, 0.368008200020995, 0.3999242999998387, 0.36087879998376593, 0.3506790999963414, 0.3887508000188973, 0.37827079999260604, 0.395228200010024, 0.36694040001020767]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.7268685000017285, 1.4967584000260103, 1.873400299984496, 1.5100730000121985, 1.458024599996861, 1.4608237000065856, 1.4706342999998014, 1.52441220000037, 1.4864618999999948, 1.4889831000182312]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.5048466000007465, 1.4521679000172298, 1.5160182999970857, 1.495718099991791, 1.4959235000133049, 1.4577977000153624, 1.4798630999866873, 1.4678222000075039, 1.8326086000015493, 1.9673086999973748]
pearson.get_trends(labeled_df.df,

[0.4734843999904115, 0.4970169000152964, 0.48292900001979433, 0.5425434999924619, 0.48305049998452887, 0.48606299998937175, 0.49230339998030104, 0.4810680000227876, 0.4684801000112202, 0.5028221000102349]
labeled_df.get_subgroup_trends_1lev([pearson])
[1.923452199989697, 1.4130474999838043, 1.4170733999926597, 1.4327660000126343, 1.418881399993552, 1.4226070999866351, 1.4079184999864083, 1.425814000016544, 1.428159000002779, 1.4024651000218]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.4189418000169098, 1.4124279000097886, 1.4197184999939054, 1.4150235999841243, 1.393400200002361, 1.4204387000063434, 1.4124375999963377, 1.4060333000088576, 1.5086977000173647, 1.580332699988503]
pearson.get_trends(labeled_df.df, 'agg_trend')
[0.7538373999996111, 0.7549055000126828, 0.8230476999888197, 1.4153221999877132, 0.9345434999850113, 0.8435400000016671, 0.8296682999935001, 0.7767494000145234, 0.9124575000023469, 1.058866899984423]
pearson_2.get_trends(labeled_df.df, 'agg_trend')
[2.1352153

[2.4990158999862615, 1.8822686999919824, 1.7608879999897908, 1.8006738999974914, 1.7125390000001062, 1.663809499994386, 1.542963999992935, 1.4516483999905176, 1.4374556000111625, 1.4218909000046551]
labeled_df.get_subgroup_trends_1lev([pearson_2])
[1.8104109000123572, 1.5037968000106048, 1.453254599997308, 1.5222080000094138, 1.739815799985081, 1.426214799983427, 1.4298069000069518, 1.4399571999965701, 1.455795100016985, 1.4250351999944542]
pearson.get_trends(labeled_df.df, 'agg_trend')
[1.060713000013493, 1.0773177999944892, 1.0943982999888249, 1.2480589999759104, 1.4713880999770481, 1.494770300021628, 1.5066548000031617, 1.4364776000147685, 1.496364500024356, 1.4873114999791142]
pearson_2.get_trends(labeled_df.df, 'agg_trend')
[3.5084098000079393, 3.365882000012789, 3.4692640000139363, 3.6045383000164293, 3.419465599989053, 4.474299900000915, 3.8648174999980256, 3.51635069999611, 3.996468500001356, 3.580541600007564]
pearson.get_trends(labeled_df.df, 'sub_trend')
[1.0858966000087094,

[1.4134529999864753, 1.4150488000013866, 1.4246007000037935, 1.4062754999904428, 1.4171128000016324, 1.4641265000100248, 1.4289937000139616, 1.4102622999926098, 1.394172800006345, 1.4073666000040248]
pearson.get_trends(labeled_df.df, 'agg_trend')
[0.5855312999919988, 0.5630412999889813, 0.5745509999978822, 0.5713396999926772, 0.5743366999959107, 0.5795384999946691, 0.595197399990866, 0.5818915999843739, 0.5684006999945268, 0.5791457000013907]
pearson_2.get_trends(labeled_df.df, 'agg_trend')
[1.6381346000125632, 1.7862736000097357, 1.6587108000239823, 1.745791299996199, 1.6883883999835234, 1.6716929000103846, 1.6949776999827009, 1.6832523999910336, 2.001895699999295, 1.6649145999981556]
pearson.get_trends(labeled_df.df, 'sub_trend')
[0.5907077000010759, 0.5903970999934245, 0.6057384999876376, 0.5774477999948431, 0.584970500000054, 0.5981648999731988, 0.5760485000209883, 0.597716299991589, 0.5896254000253975, 0.597386200010078]
pearson_2.get_trends(labeled_df.df, 'sub_trend')
[1.64953249

...

[1.793506799993338, 1.5163847000221722, 1.5109334999870043, 1.498193999985233, 1.4929811999900267, 1.4862888000207022, 1.4892983000027016, 1.5044729000073858, 1.4917382999847177, 1.4977064999984577]
pearson_2.get_trends(labeled_df.df, 'agg_trend')
[2.977295899996534, 2.921408499998506, 2.974142599996412, 2.9809813000028953, 2.9559043000044767, 3.057946199987782, 3.4219496000150684, 2.940682000014931, 3.3274097999965306, 2.9801174999738578]
pearson.get_trends(labeled_df.df, 'sub_trend')
[1.5043803000007756, 1.4924523999798112, 1.4904943000001367, 1.4848841000057291, 1.4761444999894593, 1.4761823000153527, 1.5195441000105347, 1.5291665999975521, 1.492469400021946, 1.4751245999941602]
pearson_2.get_trends(labeled_df.df, 'sub_trend')
[2.988086099998327, 2.9673986000125296, 2.9295576000004075, 2.93625470000552, 2.9697910000104457, 2.9621753999963403, 2.963698799983831, 2.9461734000069555, 2.9584146999986842, 2.9268661999958567]
independents:  15
dependents:  5
pearson.compute_correlation_ta

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "c:\users\pad39\appdata\local\programs\python\python38\lib\abc.py", line 96, in __instancecheck__
    def __instancecheck__(cls, instance):
KeyboardInterrupt: 


[1.4901294000155758, 1.4356702000077348, 1.4783046999946237, 1.740665599994827, 1.4200175000005402, 1.4321960999805015, 1.4299161999952048, 1.429190800001379, 1.4598969999933615, 1.4869682000135072]
pearson.get_trends(labeled_df.df, 'agg_trend')
[2.1326115999836475, 2.336252599983709, 2.1740726999996696, 2.114355900004739, 2.153683699987596, 2.127667000022484, 2.120511599991005, 2.122933300008299, 2.118275299988454, 2.135591600002954]
pearson_2.get_trends(labeled_df.df, 'agg_trend')


KeyboardInterrupt: 

In [11]:
# result.to_csv('data/all_data')

In [19]:
result

,times,data,function,trial,version,number_cluster,num_splitby,independent,dependent
0,0.382298,synthetic 1000,cct,0,v1,2,4,5,5
1,0.249277,synthetic 1000,cct,1,v1,2,4,5,5
2,0.248332,synthetic 1000,cct,2,v1,2,4,5,5
3,0.247554,synthetic 1000,cct,3,v1,2,4,5,5
4,0.257233,synthetic 1000,cct,4,v1,2,4,5,5
...,...,...,...,...,...,...,...,...,...
75,5.452753,synthetic 5000,get_trends (sub),5,v2,2,4,20,5
76,5.521755,synthetic 5000,get_trends (sub),6,v2,2,4,20,5
77,5.229496,synthetic 5000,get_trends (sub),7,v2,2,4,20,5
78,5.191616,synthetic 5000,get_trends (sub),8,v2,2,4,20,5


In [20]:
all_data=pd.read_csv('data/all_data')

In [21]:
final=pd.concat([all_data, result])

In [23]:
final=final.drop(['Unnamed: 0'], axis=1)

In [24]:
final

,times,data,function,trial,version,number_cluster,num_splitby,independent,dependent
0,0.360980,synthetic 1000,cct,0,v1,2,4,5,5
1,0.265732,synthetic 1000,cct,1,v1,2,4,5,5
2,0.250074,synthetic 1000,cct,2,v1,2,4,5,5
3,0.242783,synthetic 1000,cct,3,v1,2,4,5,5
4,0.251419,synthetic 1000,cct,4,v1,2,4,5,5
...,...,...,...,...,...,...,...,...,...
75,5.452753,synthetic 5000,get_trends (sub),5,v2,2,4,20,5
76,5.521755,synthetic 5000,get_trends (sub),6,v2,2,4,20,5
77,5.229496,synthetic 5000,get_trends (sub),7,v2,2,4,20,5
78,5.191616,synthetic 5000,get_trends (sub),8,v2,2,4,20,5


In [25]:
final.to_csv('data/all_data')

In [26]:
final

,times,data,function,trial,version,number_cluster,num_splitby,independent,dependent
0,0.360980,synthetic 1000,cct,0,v1,2,4,5,5
1,0.265732,synthetic 1000,cct,1,v1,2,4,5,5
2,0.250074,synthetic 1000,cct,2,v1,2,4,5,5
3,0.242783,synthetic 1000,cct,3,v1,2,4,5,5
4,0.251419,synthetic 1000,cct,4,v1,2,4,5,5
...,...,...,...,...,...,...,...,...,...
75,5.452753,synthetic 5000,get_trends (sub),5,v2,2,4,20,5
76,5.521755,synthetic 5000,get_trends (sub),6,v2,2,4,20,5
77,5.229496,synthetic 5000,get_trends (sub),7,v2,2,4,20,5
78,5.191616,synthetic 5000,get_trends (sub),8,v2,2,4,20,5


In [39]:
final=pd.read_csv('data/all_data')
final.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1'], axis=1)

,Unnamed: 0.1.1.1,times,data,function,trial,version,number_cluster,num_splitby,independent,dependent
0,0.0,0.360980,synthetic 1000,cct,0,v1,2,4,5,5
1,1.0,0.265732,synthetic 1000,cct,1,v1,2,4,5,5
2,2.0,0.250074,synthetic 1000,cct,2,v1,2,4,5,5
3,3.0,0.242783,synthetic 1000,cct,3,v1,2,4,5,5
4,4.0,0.251419,synthetic 1000,cct,4,v1,2,4,5,5
...,...,...,...,...,...,...,...,...,...,...
19035,NaN,4.826503,synthetic 5000,get_trends (sub),5,v2,4,4,25,5
19036,NaN,4.832889,synthetic 5000,get_trends (sub),6,v2,4,4,25,5
19037,NaN,4.841487,synthetic 5000,get_trends (sub),7,v2,4,4,25,5
19038,NaN,4.884433,synthetic 5000,get_trends (sub),8,v2,4,4,25,5
